### Pass Network

In [1]:
# Import Libraries
import pandas as pd
from statsbombpy import sb

In [ ]:
# Get a desired match passing 
matchID = 3923881
dfEvents = sb.events(match_id= matchID)

# Split columns
dfEvents[['location_x', 'location_y']] = dfEvents['location'].apply(pd.Series)
dfEvents[['pass_end_location_x', 'pass_end_location_y']] = dfEvents['pass_end_location'].apply(pd.Series)

# Get pass events
passDF = dfEvents[dfEvents['type'] == 'Pass']

# Filter for needed data
passNetworkDF = passDF[['id','match_id','team','period','minute','player','player_id',
                'location_x','location_y','pass_recipient','pass_recipient_id',
                'pass_end_location_x','pass_end_location_y','pass_type','pass_outcome']]

def passoutcome(row):
    if pd.isna(row['pass_outcome']):
        return 'complete'
    elif row['pass_outcome'] == 'Incomplete':
        return 'incomplete'
    else:
        return None

# Applky the function and create a new column
passNetworkDF['pass_success'] = passNetworkDF.apply(passoutcome, axis = 1)
passNetworkDF.head()

In [16]:
dfEvents[(dfEvents['shot_outcome'] == 'Goal') & (dfEvents['team'] == "Côte d'Ivoire")][['minute']]

,minute
2483,61
2486,80


In [53]:
dfEvents[(dfEvents['shot_outcome'] == 'Goal') & (dfEvents['team'] == "Nigeria")][['minute']]
#b[['minute']]

,minute
2476,37


### Full Game

In [ ]:
completedPass = passNetworkDF[passNetworkDF['pass_success'] == 'complete']

avgLocation = completedPass.groupby(['player','team']).agg({'location_x': ['mean'], 'location_y':['mean','count']})
avgLocation.columns = ['location_x','location_y','count']

In [ ]:
passBetween = completedPass.groupby(['player','pass_recipient','team']).id.count().reset_index()
passBetween.rename({'id':'pass_count'}, axis='columns', inplace=True)

In [27]:
passBetween = passBetween.merge(avgLocation, left_on = ['player','team'], right_index=True)
passBetween = passBetween.merge(avgLocation, left_on= ['pass_recipient','team'], right_index=True, suffixes=['','_end'])

In [30]:
passBetween = passBetween[passBetween['pass_count'] > 1]
#passBetween.to_csv(r'./FullPassNetwork.csv', index=False)

Get pass network before conceeeding (1-61) and after conceeding (61-end)

### Nigeria

In [45]:
n_completedPass = passNetworkDF[(passNetworkDF['pass_success'] == 'complete') & (passNetworkDF['team'] == 'Nigeria')]

# Pass network before goal
bGoalT = 61

def goalstatus(row):
    if row['minute'] < bGoalT:
        return 'beforeGoal'
    else:
        return 'afterGoal'

n_completedPass['goal_status'] = n_completedPass.apply(goalstatus, axis=1)

# Get the time for first sub
n_completedPass.head()

C:\Users\Razak\AppData\Local\Temp\ipykernel_21660\3388147685.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n_completedPass['goal_status'] = n_completedPass.apply(goalstatus, axis=1)


,id,match_id,team,period,minute,player,player_id,location_x,location_y,pass_recipient,pass_recipient_id,pass_end_location_x,pass_end_location_y,pass_type,pass_outcome,pass_success,goal_status
6,72e4bb5b-f908-4af9-afc4-0824b4e0de96,3923881,Nigeria,1,0,Ademola Lookman,3447.0,61.0,40.1,William Troost-Ekong,5455.0,35.1,52.5,Kick Off,NaN,complete,beforeGoal
7,da854004-2f92-4ff7-944b-a83198058536,3923881,Nigeria,1,0,William Troost-Ekong,5455.0,35.1,52.5,Calvin Bassey Ughelumba,43779.0,35.2,33.5,NaN,NaN,complete,beforeGoal
8,4465f67b-c718-43c2-a934-cae7d3b65c4d,3923881,Nigeria,1,0,Calvin Bassey Ughelumba,43779.0,38.3,32.9,Oluwasemilogo Adesewo Ibidapo Ajayi,3923.0,97.1,56.5,NaN,NaN,complete,beforeGoal
20,e56ea592-671c-48cd-83d8-b463893e29ab,3923881,Nigeria,1,0,Zaidu Sanusi,31949.0,27.8,15.3,Alex Iwobi,3647.0,33.8,26.2,Recovery,NaN,complete,beforeGoal
21,9b739ef7-1e63-42e6-8b5b-943e2d7ae4dc,3923881,Nigeria,1,0,Alex Iwobi,3647.0,33.8,24.8,Victor James Osimhen,11277.0,71.0,18.1,NaN,NaN,complete,beforeGoal


In [46]:
avgLocation = n_completedPass.groupby(['player','goal_status']).agg({'location_x': ['mean'], 'location_y':['mean','count']})
avgLocation.columns = ['location_x','location_y','count']

In [47]:
passBetween = n_completedPass.groupby(['player','pass_recipient','goal_status']).id.count().reset_index()
passBetween.rename({'id':'pass_count'}, axis='columns', inplace=True)

In [56]:
passBetween = passBetween.merge(avgLocation, left_on = ['player','goal_status'], right_index=True)
passBetween = passBetween.merge(avgLocation, left_on= ['pass_recipient','goal_status'], right_index=True, suffixes=['','_end'])
passBetween.head()

,player,pass_recipient,goal_status,pass_count,location_x_x,location_y_x,count_x,location_x_end,location_y_end,count_end,location_x_y,location_y_y,count_y,location_x,location_y,count
0,Ademola Lookman,Alex Iwobi,afterGoal,2,47.966667,26.483333,6,57.1200,27.1600,5,47.966667,26.483333,6,57.1200,27.1600,5
31,Frank Onyeka,Alex Iwobi,afterGoal,1,71.150000,55.125000,4,57.1200,27.1600,5,71.150000,55.125000,4,57.1200,27.1600,5
50,Oluwasemilogo Adesewo Ibidapo Ajayi,Alex Iwobi,afterGoal,1,57.500000,62.300000,2,57.1200,27.1600,5,57.500000,62.300000,2,57.1200,27.1600,5
3,Ademola Lookman,Temitayo Olufisayo Olaoluwa Aina,afterGoal,1,47.966667,26.483333,6,76.5125,61.5125,8,47.966667,26.483333,6,76.5125,61.5125,8
20,Alhassan Yusuf,Temitayo Olufisayo Olaoluwa Aina,afterGoal,1,72.680000,33.040000,5,76.5125,61.5125,8,72.680000,33.040000,5,76.5125,61.5125,8


In [49]:
#passBetween.to_csv(r'./NFullPassNetwork.csv', index=False)

Players List

In [36]:
nfullLineup = sb.lineups(matchID)['Nigeria']
cfullLineup = sb.lineups(matchID)["Côte d'Ivoire"]
fullLineup = pd.concat([nfullLineup,cfullLineup], axis=0)
players_df = fullLineup[['player_id','player_name','jersey_number','country']].reset_index(drop=True)

# Export players info
#players_df.to_csv(r'./PlayersList.csv', index=False)
players_df.head()

c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,player_id,player_name,jersey_number,country
0,3139,Chidozie Collins Awaziem,20,Nigeria
1,3447,Ademola Lookman,18,Nigeria
2,3647,Alex Iwobi,17,Nigeria
3,3708,Kelechi Promise Iheanacho,14,Nigeria
4,3806,Bright Osayi-Samuel,12,Nigeria


Match Info

In [39]:
matchDF = sb.matches(competition_id=1267, season_id=107)
matchinfo = matchDF[matchDF['match_id'] == matchID].reset_index(drop=True)

# Export
#matchinfo.to_csv(r'./Match Info.csv', index =False)
matchinfo

c:\Users\Razak\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3923881,2024-02-11,22:00:00.000,Africa - African Cup of Nations,2023,Nigeria,Côte d'Ivoire,1,2,available,...,None,8,Final,Stade Olympique Alassane Ouattara,Dahane Beida,José Vítor dos Santos Peseiro,Emerse Faé,1.1.0,2,2
